In [0]:
%run "./common_functions"

In [0]:
%run "./configuration"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

## Ingesting multiline json file

In [0]:
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from pyspark.sql.functions import lit

#### Read json file and define schema

In [0]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("stop", StringType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
])

In [0]:
# Adding multiLine=True in options
pit_stops_df = spark.read.schema(pit_stops_schema).options(multiLine=True).json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")

In [0]:
pit_stops_df = pit_stops_df \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
pit_stops_df = add_ingestion_date(pit_stops_df)

In [0]:
#overwrite_partition(pit_stops_df, "f1_processed", "pit_stops", "race_id")

merge_condition = "event.race_id = update.race_id AND event.driver_id = update.driver_id AND event.stop = update.stop"

merge_delta_data(pit_stops_df, "f1_processed", "pit_stops", processed_folder_path, merge_condition, "race_id")

In [0]:
dbutils.notebook.exit("Success")